# Applied Data Science Capstone Project

## Introduction

#### Learn about potential new office locations

The Dublin Ohio development lab office is likely to move to a new location in the next year or two.  I would like to understand venue types in particular areas of the city as well as what drive time I could expect from my home location to the new potential office locations.

## Target Audience
The target audience is primarily me but the same process and material would be usable by others in the Dublin office by providing a different source address.   

This data will be important to people in the Dublin office as we decide if we can drive into the new office or will need to look for a new job.  

----------

## Data

My data will come from a number of sources, including wikipedia, google/bing, foursquare and the USPS.

I will use:
 * wikipedia and other sites about Columbus Ohio to gather zip code and neighborhood info.  
 * foursquare to collect info on locations and venues
 * google/bing to determine drive time to each location from my home.  

The data zip code and neighborhood data will be of the format:

> Zip, neighborhood, longitude, latitude

The venue data will be of the format:

> Neighborhood, top venues 1-10

The travel data will be of the format:

> Neighborhood, travel distance to loation, travel time to location

I will use google or bing to collect the travel data.
https://msdn.microsoft.com/en-us/library/ff701717.aspx

----------

## Methodology

Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, and what machine learnings were used and why.

## Discussion

See the associated presentation.

## Results

See the associated presentation.

## Conclusion

See the associated presentation.

In [146]:
import numpy as np  # useful for many scientific computing in Python
import pandas as pd # primary data structure library
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print("Hello Capstone Project Course!")

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Hello Capstone Project Course!


Get a list of neighborhoods

In [284]:
#page_link = 'https://columbusneighborhoods.org/explore-central-ohio/'
#response = requests.get(page_link)

#page_src = response.text
#print('done')

## Initial source and location data

I was not able to find a clean mapping of neighborhood, zip, longitude and latitude to use for the columbus area.  Instead, I collected a list of neighborhoods, suburbs, cities and towns of interest in the study (i.e close to Columbus and the existing Dublin Ohio office).  I will use that list with a variety of techniques to collect the zipcode and longitude and latitude for later use.  

In [291]:
cmh_neighborhoods_str = 'Lewis Center,Powell,Delaware,Plain City,Marysville,Bexley,Brewery District,Canal Winchester,Clintonville,Columbus,Dublin,Franklin Park,Franklinton,Gahanna,German Village,Grandview Heights,Grove City,Groveport,Harrison West,Hilliard,Italian Village,King-Lincoln,Linden,Merion Village,Minerva Park,New Albany,North Linden,Northland,Olde Towne East,Reynoldsburg,Short North,University District,Upper Arlington,Victorian Village,Weinland Park,Westerville,Westgate,Whitehall,Worthington'
cmh_neighborhoods_list = cmh_neighborhoods_str.split(',')
cmh_neighborhoods_list.sort()

column_names = ['Neighborhood', 'Postcode', 'Longitude', 'Latitude'] 
df = pd.DataFrame(columns=column_names)

df['Neighborhood'] = cmh_neighborhoods_list
df.set_index('Neighborhood', inplace=True)
df.head(5)

,Postcode,Longitude,Latitude
Neighborhood,,,
Bexley,NaN,NaN,NaN
Brewery District,NaN,NaN,NaN
Canal Winchester,NaN,NaN,NaN
Clintonville,NaN,NaN,NaN
Columbus,NaN,NaN,NaN


## Zip to long/lat data

Found this fantastic data set of all US zipcodes mapped to a longitude and latitude location.  Loading it up in case I need it later.  

In [167]:
allzip_url = 'https://gist.github.com/erichurst/7882666/raw/5bdc46db47d9515269ab12ed6fb2850377fd869e/US%2520Zip%2520Codes%2520from%25202013%2520Government%2520Data'
df_zip = pd.read_csv(allzip_url)
df_zip.head()

,ZIP,LAT,LNG
0,601,18.180555,-66.749961
1,602,18.361945,-67.175597
2,603,18.455183,-67.119887
3,606,18.158345,-66.932911
4,610,18.295366,-67.125135


In [168]:
df_zip.columns = ('Postcode', 'Latitude', 'Longitude')
df_zip['Postcode'] = df_zip['Postcode'].astype(str)
df_zip.set_index('Postcode', inplace=True)
df_zip.head()

,Latitude,Longitude
Postcode,,
601,18.180555,-66.749961
602,18.361945,-67.175597
603,18.455183,-67.119887
606,18.158345,-66.932911
610,18.295366,-67.125135


In [40]:
# Test it out and see if it has the right data for a local location
df_zip.loc["43015"]

Longitude    40.297786
Latitude    -83.060181
Name: 43015, dtype: float64

-----
### Start enriching the neighborhood list with data from sources now.  

Since I don't have a way to good data source to map the neighborhood to a zip, I am going to use foursquare and do a lookup by name of the neighborhood and loop through the businesses for a zip code.  That zip code will become the zip code for the neighborhood.  


In [65]:
CLIENT_ID = '1IVL5ODVU4BG2GIPGM23IA1UQAOKYCQUHS0G1X3AQ0WXZ4SQ' # your Foursquare ID
CLIENT_SECRET = 'O1YZXQ05MNOUOV2ERW0YXZ30UK5C1SGH2TS5EF415HPG5JCA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1IVL5ODVU4BG2GIPGM23IA1UQAOKYCQUHS0G1X3AQ0WXZ4SQ
CLIENT_SECRET:O1YZXQ05MNOUOV2ERW0YXZ30UK5C1SGH2TS5EF415HPG5JCA


Use Foursquare to lookup the neighborhood name and find the first business with a zip.  This becomes our zip for the neighborhood.

In [127]:
def get_zip(address):
    LIMIT = 10 # limit of number of venues returned by Foursquare API
    radius = 5 # define radius
    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&near={}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, VERSION, 
        address, radius, LIMIT)

    results = requests.get(url).json()['response']

    if ('venues' not in results): return('0')
    
    for v in results['venues']:
        if ('postalCode' in v['location']): 
            zipcode = v['location']['postalCode']
            break
    
    #print('The zip for {} is {}.'.format(address, zip))
    return(zipcode)


### Get a zip for each neighborhood

Walk the dataframe and collect the zip for each neighborhood listed.

If we find a zip code for the neighborhood, also add the longitude and latitude for that zip based on the df_zip data retrieved earlier.  At the end of this step, we will have neighborhood, zip, long and lat in a dataframe.  


In [292]:
for name in cmh_neighborhoods_list:
    foundzip = get_zip(name + " Ohio")

    if (foundzip != '0'):
        print(name + ' : ' + foundzip)
        df.loc[name,'Postcode'] = foundzip
        df.loc[name,'Longitude'] = df_zip.loc[foundzip, 'Longitude']
        df.loc[name,'Latitude'] = df_zip.loc[foundzip, 'Latitude']

Bexley : 43209
Canal Winchester : 43110
Columbus : 43215
Delaware : 43015
Dublin : 43017
Franklin Park : 43203
Gahanna : 43230
Grandview Heights : 43212
Grove City : 43123
Groveport : 43125
Hilliard : 43026
Lewis Center : 43035
Linden : 43211
Marysville : 43040
Minerva Park : 43231
New Albany : 43054
Plain City : 43064
Powell : 43065
Reynoldsburg : 43068
Upper Arlington : 43212
Weinland Park : 43201
Westerville : 43081
Whitehall : 43213
Worthington : 43085


In [293]:
df.dropna(inplace=True)
df.head(5)

,Postcode,Longitude,Latitude
Neighborhood,,,
Bexley,43209,-82.9307,39.9536
Canal Winchester,43110,-82.8001,39.8267
Columbus,43215,-83.013,39.9669
Delaware,43015,-83.0602,40.2978
Dublin,43017,-83.129,40.1107


## Visualize with a map

Now that we have the data, lets see how things look on the map.  The current office is in Dublin Ohio.


In [288]:
geolocator = Nominatim(user_agent='python')

address = 'Columbus, Ohio'
location = geolocator.geocode(address)
cmh_lat = location.latitude
cmh_long = location.longitude
print('The geograpical coordinate of Columbus are {}, {}.'.format(cmh_lat, cmh_long))

# Current office location
office_lat = 40.076397
office_long = -83.139124

The geograpical coordinate of Columbus are 39.9622601, -83.0007065.


In [295]:
df.reset_index(inplace=True)
df.head()

,Neighborhood,Postcode,Longitude,Latitude
0,Bexley,43209,-82.9307,39.9536
1,Canal Winchester,43110,-82.8001,39.8267
2,Columbus,43215,-83.013,39.9669
3,Delaware,43015,-83.0602,40.2978
4,Dublin,43017,-83.129,40.1107


## Columbus area with potential office move destinations

At this point, we can see the overall city and outlying cities/suberbs that are potential new office locations. 

The current office location is in red.


In [299]:
# create map of New York using latitude and longitude values
map_cmh = folium.Map(location=[cmh_lat, cmh_long], zoom_start=10)
print("{} {} {}".format("cmh", cmh_lat, cmh_long))

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    #print("{} {} {}".format(neighborhood, lat, lng))
    label = folium.Popup(neighborhood, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cmh)  
    
folium.CircleMarker(
        [office_lat, office_long],
        radius=5,
        popup='Current Office Location',
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cmh)      
    
map_cmh

cmh 39.9622601 -83.0007065


## Determine top venues for each area above

Using techniques from the lab, I will lookup and produce the top venues in each area for later analysis.  



In [303]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [304]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)

        #print(url)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        if (len(results) == 0):
           venues_list.append([(name, lat, lng, 'None', lat, lng)])   
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
            
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [305]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [306]:
# type your answer here
cmh_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )
cmh_venues.shape


Bexley
Canal Winchester
Columbus
Delaware
Dublin
Franklin Park
Gahanna
Grandview Heights
Grove City
Groveport
Hilliard
Lewis Center
Linden
Marysville
Minerva Park
New Albany
Plain City
Powell
Reynoldsburg
Upper Arlington
Weinland Park
Westerville
Whitehall
Worthington


(228, 7)

In [307]:
# one hot encoding
cmh_onehot = pd.get_dummies(cmh_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
cmh_onehot['Neighborhood'] = cmh_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [cmh_onehot.columns[-1]] + list(cmh_onehot.columns[:-1])
cmh_onehot = cmh_onehot[fixed_columns]

cmh_onehot.head()

,Neighborhood,American Restaurant,Arcade,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,...,Spa,Speakeasy,Steakhouse,Thrift / Vintage Store,Tourist Information Center,Turkish Restaurant,Video Store,Wine Bar,Wine Shop,Wings Joint
0,Bexley,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bexley,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,Bexley,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bexley,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bexley,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [308]:
cmh_grouped = cmh_onehot.groupby('Neighborhood').mean().reset_index()
cmh_grouped.head()

,Neighborhood,American Restaurant,Arcade,Asian Restaurant,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,...,Spa,Speakeasy,Steakhouse,Thrift / Vintage Store,Tourist Information Center,Turkish Restaurant,Video Store,Wine Bar,Wine Shop,Wings Joint
0,Bexley,0.0,0.0,0.0,0.000000,0.055556,0.055556,0.111111,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.055556,0.0,0.0,0.0,0.000000
1,Canal Winchester,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
2,Columbus,0.0,0.0,0.0,0.043478,0.000000,0.000000,0.000000,0.043478,0.0,...,0.0,0.0,0.0,0.0,0.043478,0.000000,0.0,0.0,0.0,0.043478
3,Delaware,0.0,0.0,0.0,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
4,Dublin,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000


In [350]:
num_top_venues = 5

for hood in cmh_grouped['Neighborhood']:
    #print("----"+hood+"----")
    temp = cmh_grouped[cmh_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

In [351]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [353]:
num_top_venues = 4

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = cmh_grouped['Neighborhood']

for ind in np.arange(cmh_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cmh_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,Bexley,Sandwich Place,Bank,Pizza Place,Pharmacy
1,Canal Winchester,Wings Joint,Convenience Store,Deli / Bodega,Diner
2,Columbus,Rock Club,Deli / Bodega,Park,Business Service
3,Delaware,Grocery Store,BBQ Joint,Garden Center,Liquor Store
4,Dublin,Football Stadium,Baseball Field,Wings Joint,Food & Drink Shop
5,Franklin Park,Park,Plaza,Bank,Chinese Restaurant
6,Gahanna,Beach Bar,Chinese Restaurant,Speakeasy,Food
7,Grandview Heights,Bar,Pizza Place,American Restaurant,Italian Restaurant
8,Grove City,Wings Joint,Convenience Store,Deli / Bodega,Diner
9,Groveport,Wings Joint,Convenience Store,Deli / Bodega,Diner


In [354]:
cmh_merged = df
cmh_merged = cmh_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
cmh_merged.columns

cmh_merged.head() # check the last columns!

,Neighborhood,Postcode,Longitude,Latitude,Drive Distance,Drive Time,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,Bexley,43209,-82.9307,39.9536,46.695,1887,Sandwich Place,Bank,Pizza Place,Pharmacy
1,Canal Winchester,43110,-82.8001,39.8267,56.794,2284,Wings Joint,Convenience Store,Deli / Bodega,Diner
2,Columbus,43215,-83.013,39.9669,37.285,1699,Rock Club,Deli / Bodega,Park,Business Service
3,Delaware,43015,-83.0602,40.2978,22.768,1237,Grocery Store,BBQ Joint,Garden Center,Liquor Store
4,Dublin,43017,-83.129,40.1107,21.689,1060,Football Stadium,Baseball Field,Wings Joint,Food & Drink Shop


# Driving distances

A major consideration for office location is how difficult the communte will be.  Today the office, is located in Dublin Ohio just off the major ring freeway.  

The following work is to determine driving distances and times to the various locations around the city.  


## Starting location for driving distance

This analysis uses my home address as the source driving location (waypoint 0) to determine the driving distances.  In a more detailed analysis, this could be done with a listing of employee addreses from the office to show how this would impact different employees.  



In [197]:
# My home longitude and latitude
# I used mapquest manually to look this up
#  https://developer.mapquest.com/documentation/tools/latitude-longitude-finder/
#
home_long = '-83.15789'
home_lat = '40.195511'

In [280]:
def getDistance(names, latitudes, longitudes):
    #dist_column_names = ['Neighborhood', 'Drive Distance', 'Drive Time'] 
    #distance_df = pd.DataFrame(columns=dist_column_names)

    distance_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
        distance_url = 'http://dev.virtualearth.net/REST/V1/Routes?wp.0={},{}&wp.1={},{}&optmz=timeWithTraffic&key=Alrvl5RASAnV_-OuDDnGzeHBsgTu3k-sdaySCwoQyWOUepEx3XJuWsPFsIb7LJrp'.format(
home_lat, home_long, lat, long)
            
        # make the GET request
        results = requests.get(distance_url).json()['resourceSets'][0]['resources'][0]

        distance_list.append([name, results['travelDistance'], results['travelDuration']])
        #distance_df.append([name, results['travelDistance'], results['travelDuration']], ignore_index=True)

    distance_df = pd.DataFrame(distance_list)
    distance_df.columns = ['Neighborhood', 'Drive Distance', 'Drive Time']        
    
    return(distance_df)
   

## Some limitations from this driving API

I used this driving API from bing.  Had to sign up for a free account but it is limited.

http://dev.virtualearth.net

This API doesn't support (or at least not for free) the ability to specify the time of day and day of week so will work with the time and distance when the query is run.  Distance shouldn't change significantly unless traffic would force a different route but that won't be accounted for here.  In a real analysis, we would pay for the Google API that provides all this.

In [325]:
#Determine all the distances from the source point to each neighborhood in the list.

distances_df = getDistance(df['Neighborhood'], df['Latitude'], df['Longitude'])
print('Done')

Done


In [327]:
distances_df.sort_values(by=['Drive Time']).head(10)

,Neighborhood,Drive Distance,Drive Time
17,Powell,15.614,935
11,Lewis Center,16.945,1043
4,Dublin,21.689,1060
21,Westerville,22.324,1189
3,Delaware,22.768,1237
23,Worthington,23.471,1272
13,Marysville,23.072,1299
15,New Albany,27.005,1309
16,Plain City,24.017,1321
14,Minerva Park,27.464,1351


## Bring the distance info back in with venues

In [355]:
cmh_merged['Drive Distance'] = distances_df['Drive Distance']
cmh_merged['Drive Time'] = distances_df['Drive Time']

In [359]:
cmh_merged
final_df = cmh_merged.drop(columns="Postcode")
final_df.drop(columns="Longitude", inplace=True)
final_df.drop(columns="Latitude", inplace=True)
#final_df.drop(columns="Latitude", inplace=True)

final_df.sort_values(by=['Drive Time']).head(10)


,Neighborhood,Drive Distance,Drive Time,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
17,Powell,15.614,935,Café,Wings Joint,Food & Drink Shop,Deli / Bodega
11,Lewis Center,16.945,1043,Park,Restaurant,Baseball Field,Wings Joint
4,Dublin,21.689,1060,Football Stadium,Baseball Field,Wings Joint,Food & Drink Shop
21,Westerville,22.324,1189,Park,Wings Joint,Food & Drink Shop,Deli / Bodega
3,Delaware,22.768,1237,Grocery Store,BBQ Joint,Garden Center,Liquor Store
23,Worthington,23.471,1272,Furniture / Home Store,Bakery,Restaurant,Playground
13,Marysville,23.072,1299,Pizza Place,Dog Run,Wings Joint,Food & Drink Shop
15,New Albany,27.005,1309,Food Truck,Coffee Shop,Liquor Store,Food Court
16,Plain City,24.017,1321,Wings Joint,Convenience Store,Deli / Bodega,Diner
14,Minerva Park,27.464,1351,Golf Course,Park,Pool,Garden Center


In [346]:
final_df